In [17]:
import pandas as pd
import numpy as np
from surprise import SVD
from surprise import dump
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, train_test_split

In [18]:
def create_user():
    user_name = input('Please enter your name: ')
    print("\n Welcome {}! I'm going to recommend some books just for you.\n".format(user_name))
    user_id = max(df.user_id) + 1
    return user_name, user_id

In [19]:
def valid_entries(user_entry, valid_entries):
    if user_entry not in valid_entries:
        return False
    else:
        return True

In [20]:
def check_favorite_authors(df):
    valid_yes_no = ['Y', 'N', 'YES', 'NO']
    is_valid = False
    while is_valid == False:
        favorite_author_ask = (input("Do you have any favorite authors who may have recently written books? (Yes/No) ")).upper()
        if valid_entries(favorite_author_ask, valid_yes_no) == False:
            print("Sorry, that isn't a valid entry, please try again. \n")
        else:
            is_valid = True
            if favorite_author_ask in ['Y', 'YES']:
                favorite_author_ask = True
            else:
                favorite_author_ask = False
    if favorite_author_ask == True:
        keep_checking = True
        while keep_checking == True:
            favorite_author = (input("Name an author, I will check to see if they wrote anything on the top list."))
            if favorite_author.title() in book_reviews.author.unique():
                print("{} has written the following books: \n".format(favorite_author))
                for i in list(df.title.loc[df.author == favorite_author].unique()):
                    print(i + "\n")
            else:
                print("Sorry, {} has not written any books that were recently trending on our lists.")
            is_valid = False
            while is_valid == False:
                check_again = input("Would you like to check another author? (Yes/No)").upper()
                if valid_entries(check_again, valid_yes_no) == False:
                    print("Sorry, that isn't a valid entry, please try again. \n")
                else:
                    is_valid = True
            if check_again in ['Y', 'Yes']:
                keep_checking = True
            else:
                keep_checking = False

In [21]:
def num_reviews():
    x = False
    while x == False:
        num_reviews = input('How many books would you like to review? \n')
        try:
            if 0< int(num_reviews) <=10:
                x = True
                return int(num_reviews)
                break
        except:
            print('Invalid entry. Please try again.')

In [22]:
def new_user_ratings(df, num_reviews, user_name, user_id):
    user_id = new_user_id
    user_name = new_user_name
    ratings_list = []
    while num_reviews >0: 
        book = book_info_df.sample(1)
        rating = input('How do you rate this book on a scale of 1 to 5? If you are unfamiliar with this title press n. \n')
        print(book.title.item() + ' by ' + book.author.item())
        if rating == 'n':
            continue
        else:
            rating_dict = {'user_id':user_id, 'isbn':book.isbn.item(), 'rating':rating}
            ratings_list.append(rating_dict)
            num_reviews -= 1
    return ratings_list

In [23]:
def new_recommendations(df, new_ratings):
    df = df[['user_id', 'isbn', 'rating']]
    new_ratings = pd.DataFrame(new_ratings)[['user_id', 'isbn', 'rating']]
    new_df = pd.concat([df, new_ratings]).reset_index(drop=True)
    reader = Reader(rating_scale=(1,5))
    data = Dataset.load_from_df(new_df,reader)
    train, test = train_test_split(data, test_size=.2)
    model = SVD(n_epochs=17, lr_all=.015, reg_all=.125, n_factors=17)
    model.fit(train)
    preds = model.test(test)
    user_id = new_ratings.user_id[0]
    book_list = []
    for x in new_df.isbn.unique():
        book_list.append((x, model.predict(user_id,x)[3]))
    ranked_books = sorted(book_list, key=lambda x: x[1], reverse=True)
    return ranked_books

In [24]:
def num_recs():
    x = False
    while x == False:
        num_recs = input('How many books would you like to be recommended? \n')
        try:
            if 0< int(num_recs) <=10:
                x = True
                return int(num_recs)
                break
        except:
            print('Invalid entry. Please try again.')
    return num_recs

In [25]:
def recs_to_return(recs, df, num_recs):
    df_reduced = df[['title', 'author', 'isbn']].drop_duplicates()
    for i, v in enumerate(recs):
        book = df_reduced.loc[df_reduced.isbn == v[0]]
        print('Recommendation # ' + str(i+1) + ': ' + book.title.item() + ' by ' + book.author.item() + 
              ' \n https://isbnsearch.org/isbn/{}'.format(book.isbn.item())) 
        num_recs-=1
        if num_recs == 0:
            break